# Klasifikace objektů
Cvičení je zaměřené na jednotlivé metody klasifikace objektů od jednodušších pomocí popisných tvarových charakteristik až po sofostikované neuronové sítě.

## Tvarové charakteristiky
...

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb
from collections import OrderedDict

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='new_functions'>Nové funkce pro daný notebook.</a> Změna rozlišení obrazu na zvolené rozlišení, logický AND dvou obrazů a automatický ořez binárního objektu pomocí jeho bounding boxu.

In [ ]:
def resize(im, size):
    return cv2.resize(im, size, cv2.INTER_AREA)

def logical_and(bin_im, bin_mask):
    return cv2.bitwise_and(bin_im, bin_mask)

def crop_by_bounding_rect(img_bin):
    _, contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tl_x, tl_y, w, h = cv2.boundingRect(contours[0])
    return crop(img_bin, tl_x, tl_y, tl_x+w, tl_y+h)

Vytvoří databázi popisných charakteristik tvarů v dostupné složce ze souborů obrázků.

In [ ]:
def create_shape_db(folder, functions):
    db = OrderedDict()
    path, dirs, files = next(os.walk(folder))

    for file in files:
        image = cv2.imread(path + file, 0)

        shape_descriptions = list()
        for func in functions:
            shape_descriptions.append(func(image))

        key = file.split('.')[0]
        db[key] = np.array(shape_descriptions)
    return db

Třída pro práci s popisnými charakteristikami tvarů a jednotlivé dílčí funkce.

In [ ]:
# Dimensionless descriptors
class ShapeDescriptors:
    def form_factor(area, perimeter):
        return (4 * np.pi * area) / (perimeter * perimeter)
    
    def roundness(area, max_diameter):
        return (4 * area) / (np.pi * max_diameter * max_diameter)
    
    def aspect_ratio(min_diameter, max_diameter):
        return min_diameter / max_diameter;
    
    def convexity(perimeter, convex_perimeter):
        return convex_perimeter / perimeter
    
    def solidity(area, convex_area):
        return area / convex_area
    
    def compactness(area, max_diameter):
        return np.sqrt(4 / np.pi * area) / max_diameter;
        
    def extent(area, bounding_rectangle_area):
        return area / bounding_rectangle_area;

# Špičatost
def form_factor(bin_im):
    _, _, conts = contours(bin_im)
    return ShapeDescriptors.form_factor(cv2.contourArea(conts[0]), cv2.arcLength(conts[0], True))

# Kulatost
def roundness(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    _,radius = cv2.minEnclosingCircle(conts[0])
    r = ShapeDescriptors.roundness(area, 2*radius)
    if r > 1: r = 1
    return r

# Poměr stran
def aspect_ratio(bin_im):
    _, _, conts = contours(bin_im)
    dims = cv2.minAreaRect(conts[0])[1]
    min_diameter = min(dims)
    max_diameter = max(dims)
    return ShapeDescriptors.aspect_ratio(min_diameter, max_diameter)
    
# Konvexita, vypouklost
def convexity(bin_im):
    _, _, conts = contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    per = cv2.arcLength(conts[0], True)
    conv_per = cv2.arcLength(hull, True)
    r = ShapeDescriptors.convexity(per, conv_per)
    if r > 1: r = 1
    return r 

# Plnost, celistvost
def solidity(bin_im):
    _, _, conts = contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    area = cv2.contourArea(conts[0])
    conv_area = cv2.contourArea(hull)
    r = ShapeDescriptors.solidity(area, conv_area)
    if r > 1: r = 1
    return r 
    
# Kompaktnost, hutnost
def compactness(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    max_diameter = max(cv2.minAreaRect(conts[0])[1])
    r = ShapeDescriptors.compactness(area, max_diameter)
    if r > 1: r = 1
    return r 
    
# Dosah, rozměrnost
def extent(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    w, h = cv2.minAreaRect(conts[0])[1]
    return ShapeDescriptors.extent(area, w*h)

---

Seznam předchozích funkcí pro přehlednost:
- [`connect_camera(...)`](../svz.ipynb#connect_functions)


- [`show_images(...)`](../svz.ipynb#show_functions)
- [`plot_images(...)`](../svz.ipynb#show_functions)


- [`to_gray(...)`](../svz.ipynb#preprocessing_functions)
- [`to_hsv(...)`](../svz.ipynb#preprocessing_functions)
- [`negative(...)`](../svz.ipynb#preprocessing_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)


- [`segmentation_one_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_auto_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_two_thresholds(...)`](../svz.ipynb#segmentation_functions)
- [`contours(...)`](../svz.ipynb#segmentation_functions)


- [`filtration_median(...)`](../svz.ipynb#filtration_functions)
- [`fill_holes(...)`](../svz.ipynb#filtration_functions)


- [`ocr(...)`](../svz.ipynb#ocr)


- [`to_intensity(...)`](../svz.ipynb#others)


- [`warp_to_cartesian(...)`](../svz.ipynb#cart_polar_functions)
- [`warp_to_polar(...)`](../svz.ipynb#cart_polar_functions)
- [`rotate(...)`](../svz.ipynb#cart_polar_functions)


- [`artificial_circle_image(...)`](../svz.ipynb#artificial_funtions)

### Úkol
Automaticky poznejte, o který předmět z databáze se jedná. Segmentujte vhodně snímaný objekt z obrazu, následně ho oceňte pomocí tvarových charakteristik a srovnejte z databází. 

#### 1) Vytvořte databázi popisných charakteristik tvarů
Ve složce `db` jsou k dispozici obrázky objektů. Vytvořte databázi jejich popisných charakteristik s využitím takových funkcí, které máte k dispozici.

![](db.png)

In [ ]:
folder = '' ###
functions = [] ###

# Vytvoří databázi
db = create_shape_db(folder, functions)

#### 2) Nasnímejte vlastní objekty
S využitím jakéhokoli nástroje nasnímejte vlastní obrazová data objektů. Využijte k tomu zadní osvětlení (backlight). 

**Poznámka:** Nasnímejte také obrázek samotného pozadí s backlightem. Bude se hodit pro zjednodušení segmentace.

In [ ]:
name = '' ###
background_name = '' ###

image = cv2.imread(name)
background = cv2.imread(background_name)

#### 3) Segmentujte pozadí
S využitím jednoduchých funkcí vytvořte binární masku nasnímaného pozadí zadního nasvícení. Bude dále využita k segmentaci složitější scény.

In [ ]:
def binary_mask_from_background(background_image):
    ###
    
    
    return mask

In [ ]:
mask = binary_mask_from_background(background)
show_images(mask)

#### 4) Segmentujte objekty
S využitím již známých a [nově přidaných](#new_functions) funkcí vytvořte algoritmus segmentace objektu z obrazu. Cílem je vytvořit takový algoritmus, který z obrazu vytvoří jeho siluetu ořízlou pouze na její velikost (viz ukázka databáze z [Úkolu 1](#1\)-Vytvořte-databázi-popisných-charakteristik-tvarů)). Využijte stále zadního nasvícení a již segmentovaného pozadí z minulého úkolu.

In [ ]:
def segment_object(image, background):
    ### 
    
    
    return binary_object_cropped

In [ ]:
binary_object = segment_object(image, mask)
show_images(binary_object)

#### 5) Spočtěte popisné charakteristiky
U vašeho vytvořeného segmentovaného ořízlého binárního objektu spočtěte popisné tvarové charakteristiky.

In [ ]:
shape_descriptions = list()

### Postupně nebo automaticky
###


    
# Konverze do np.array formátu
shape_descriptions = np.array(shape_descriptions)

#### 6) Zjistěte, o který objekt z databáze se jedná

In [ ]:
results = list()
for key in db:
    results.append(np.linalg.norm(db[key] - shape_descriptions))

result = min(results)
result_key = list(db.keys())[np.argmin(results)]

print('Jedná se o objekt: ' + result_key)
plot_images(binary_object)